In [1]:
import numpy as np
import pandas as pd
import json
from itertools import combinations
import itertools
import random
from collections import defaultdict
import os
import shutil

In [2]:
# Read in the dataset of all complexes
all_complexes = pd.read_json("allComplexes.json")
all_complexes.head(5)

FileNotFoundError: File allComplexes.json does not exist

In [3]:
# Read in the dataset of functional complex groups
fcg = pd.read_csv("fcg.txt", sep = "\t")
fcg.head(5)

,Functional Complex Group,Root,category_name,FCG assoc. Gene Ontology ID,ComplexID,ComplexName,Organism,Synonyms,Cell line,Subunits(UniProt IDs),...,GO description,FunCat ID,FunCat description,subunits(Gene name),Subunits comment,PubMed ID,Complex comment,SWISSPROT organism,subunits(Gene name syn),subunits(Protein name)
0,Cell adhesion mediated by integrin,Signaling molecules and interaction,Signaling molecules and interaction,GO:0098609,2350,ITGAV-ITGB5 complex,Human,alphaV-beta5 integrin complex,placenta cells,P06756;P06756;P06756;P18084;P18084;P18084,...,protein binding;cell adhesion;integrin complex...,16.01;34.07;16.01;34.07,protein binding;cell adhesion;protein binding;...,ITGAV;ITGAV;ITGAV;ITGB5;ITGB5;ITGB5,NaN,NaN,NaN,Homo sapiens (Human),MSK8 VNRA VTNR,Integrin alpha-V
1,Cell adhesion mediated by integrin,Signaling molecules and interaction,Signaling molecules and interaction,GO:0098609,2354,ITGAV-ITGB6 complex,Human,alphaV-beta6 integrin complex,SW480 cells,P06756;P06756;P06756;P18564;P18564;P18564,...,protein binding;integrin complex;cell adhesion...,16.01;34.07;16.01;34.07,protein binding;cell adhesion;protein binding;...,ITGAV;ITGAV;ITGAV;ITGB6;ITGB6;ITGB6,NaN,NaN,"Cells expressing alpha(v)beta6 attach, but les...",Homo sapiens (Human),MSK8 VNRA VTNR,Integrin alpha-V
2,Cell adhesion mediated by integrin,Signaling molecules and interaction,Signaling molecules and interaction,GO:0098609,2381,ITGA2B-ITGB3 complex,Human,beta2B-beta3 integrin complex,in vitro,P05106;P05106;P08514;P08514,...,integrin complex;cell adhesion;integrin comple...,34.07;34.07,cell adhesion;cell adhesion,ITGB3;ITGB3;ITGA2B;ITGA2B,NaN,NaN,NaN,Homo sapiens (Human),GP3A,Integrin beta-3
3,Cell adhesion mediated by integrin,Signaling molecules and interaction,Signaling molecules and interaction,GO:0098609,2385,ITGA5-ITGB4 complex,Human,alpha5-beta4 integrin complex,fibroblasts GM334 cells,P05556;P05556;P08648;P08648,...,cell adhesion;integrin complex;cell adhesion;i...,34.07;34.07,cell adhesion;cell adhesion,ITGB1;ITGB1;ITGA5;ITGA5,NaN,NaN,NaN,Homo sapiens (Human),"FNRB, MDF2, MSK12",Integrin beta-1
4,Cell adhesion mediated by integrin,Signaling molecules and interaction,Signaling molecules and interaction,GO:0098609,2406,ITGA3-ITGB1 complex,Human,alpha3-beta1 integrin compex,fibroblast GM3349 cells,P05556;P05556;P26006;P26006,...,cell adhesion;integrin complex;cell adhesion;i...,34.07;34.07,cell adhesion;cell adhesion,ITGB1;ITGB1;ITGA3;ITGA3,NaN,NaN,NaN,Homo sapiens (Human),"FNRB, MDF2, MSK12",Integrin beta-1


In [4]:
# Data pre-processing------
# All complexes' subunit genes
allGenes = all_complexes[all_complexes['Organism'] == 'Human']['subunits(Gene name)']
allGenes = allGenes.tolist()
allHumanComplexes = all_complexes[all_complexes['Organism'] == 'Human']

# only fcg
fcgGenes = fcg[fcg['Organism'] == 'Human']['subunits(Gene name)']
fcgGenes = fcgGenes.tolist()
fcgHumanComplexes = fcg[fcg['Organism'] == 'Human']
print(allGenes[:5])
print(fcgGenes[:5])

['BCL6;HDAC4', 'BCL6;HDAC5', 'BCL6;HDAC7', 'EP300;CREBBP;KAT2B;NCOA3', 'SMC2;NCAPH;NCAPD2;NCAPG;SMC4']
['ITGAV;ITGAV;ITGAV;ITGB5;ITGB5;ITGB5', 'ITGAV;ITGAV;ITGAV;ITGB6;ITGB6;ITGB6', 'ITGB3;ITGB3;ITGA2B;ITGA2B', 'ITGB1;ITGB1;ITGA5;ITGA5', 'ITGB1;ITGB1;ITGA3;ITGA3']


In [5]:
# Positive Control List Generation------
def PositiveControlGeneration(df, geneList):
    
    eachComplex_dict = {}
    
    for i in range(len(df)):
        complexID = df.iloc[i]['ComplexID']
        genesInEachComplex = geneList[i].split(";")
        genesInEachComplex = list(set(genesInEachComplex))
        howManyGenes = len(genesInEachComplex)
        
        if howManyGenes > 2:
            random.shuffle(genesInEachComplex)
            genePairList = []
            
            if howManyGenes % 2 == 0:
                
                numGenePair = howManyGenes // 2

            else: 
                numGenePair = (howManyGenes - 1) // 2
            
                
            for j in range(numGenePair):
                genePairList.append([genesInEachComplex[2*j], genesInEachComplex[2*j + 1]])
            eachComplex_dict[complexID] = genePairList
                
        
        elif howManyGenes == 2:
            eachComplex_dict[complexID] = [genesInEachComplex]
        
        else:
            eachComplex_dict[complexID] = []
            
    return eachComplex_dict

        

In [6]:
# Positive Control Lists Calling
FirstPositiveControl = PositiveControlGeneration(allHumanComplexes, allGenes)
SecondPositiveControl = PositiveControlGeneration(allHumanComplexes, allGenes)
ThirdPositiveControl = PositiveControlGeneration(allHumanComplexes, allGenes)
FourthPositiveControl = PositiveControlGeneration(allHumanComplexes, allGenes)
FifthPositiveControl = PositiveControlGeneration(allHumanComplexes, allGenes)

In [7]:
# Create a matrix to who whether two distinct genes are in the same complexes------
# Create a list of strings of all unique genes------
rawGeneList = [string.split(';') for string in allGenes]
print(rawGeneList[:5])
flattenGeneList = list(itertools.chain.from_iterable(rawGeneList))
flattenGeneList = list(set(flattenGeneList))
print(flattenGeneList[:5])
len(flattenGeneList)

[['BCL6', 'HDAC4'], ['BCL6', 'HDAC5'], ['BCL6', 'HDAC7'], ['EP300', 'CREBBP', 'KAT2B', 'NCOA3'], ['SMC2', 'NCAPH', 'NCAPD2', 'NCAPG', 'SMC4']]
['MRTFA', 'PLIN3', 'PLXNB1', 'TSG101', 'PRKAG1']


4424

In [8]:
# Universal truth matrix of positive interacting subunit-----
nrow = len(flattenGeneList)
ncol = len(flattenGeneList)

rows = [0 for number in range(nrow**2)]
values = np.array(rows)
matrix = values.reshape(-1, nrow)
matrix.shape == (nrow, ncol)
print(matrix.shape)
matrix

(4424, 4424)


array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [9]:
# Create a dictionary that contains all unique human genes in one complex------
oneComplexGenes = dict()
for a in range(len(allHumanComplexes)):
    complex_id = int(allHumanComplexes.iloc[a]['ComplexID'])
    whatGenesinThatComplex = allHumanComplexes.iloc[a]['subunits(Gene name)']
    whatGenesinThatComplex = whatGenesinThatComplex.split(';')
    whatUniqueGenesinThatComplex = list(set(whatGenesinThatComplex))
    oneComplexGenes[complex_id] = whatUniqueGenesinThatComplex

In [10]:
# Check wheter all unique human gene indice are included in the matrix-----
def flatten_dict_to_list(d):
    return [item for sublist in d.values() for item in sublist]

testComplexGenes = flatten_dict_to_list(oneComplexGenes)
len(list(set(testComplexGenes))) == nrow

True

In [11]:
# Create a dictionary that summaries all associated genes with a gene
associatedGenes = dict()
for i in flattenGeneList:
    associatedList = []
    for j in oneComplexGenes.keys():
        if i in oneComplexGenes[j]:
            associatedGenesInThisComplex = [item for item in oneComplexGenes[j] if item != i]
            associatedList.extend(associatedGenesInThisComplex)
            associatedGenes[i] = list(set(associatedList))

In [12]:
# Finally update the matrix If it is the same gene or they are in the same Complexes, then assign 1
for gene1_index in range(len(flattenGeneList)):
    for gene2_index in range(len(flattenGeneList)):
        gene1 = flattenGeneList[gene1_index]
        gene2 = flattenGeneList[gene2_index]
        if gene1 == gene2:
            matrix[gene1_index, gene2_index] = 1
        else:
            gene1associatedGenes = associatedGenes.get(gene1, [])

            if gene2 in gene1associatedGenes:
                matrix[gene1_index, gene2_index] = 1
            else:
                matrix[gene1_index, gene2_index] = 0

matrix # but this can only check whether they are in the same complex, not the final list for selecting negative control

array([[1, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 1]])

In [13]:
np.count_nonzero(matrix == 0)

19475148

In [14]:
# Convert the Positive Control into a dataframe which is convenient for converting into a csv------
def flatten_gene_dict(gene_dict):
    listGenePairs = [gene_pair for sublist in gene_dict.values() for gene_pair in sublist]
    df = pd.DataFrame(listGenePairs, columns = ['Gene1', 'Gene2'])
    return df

FirstPositiveControlDF = flatten_gene_dict(FirstPositiveControl)
print(len(FirstPositiveControlDF))
FirstPositiveControlDF.head(10)

6534


,Gene1,Gene2
0,HDAC4,BCL6
1,HDAC5,BCL6
2,BCL6,HDAC7
3,EP300,NCOA3
4,KAT2B,CREBBP
5,NCAPG,SMC2
6,NCAPH,SMC4
7,HPS1,HPS4
8,HPS3,HPS6
9,CDS1,MUS81


In [15]:
SecondPositiveControlDF = flatten_gene_dict(SecondPositiveControl)
ThirdPositiveControlDF = flatten_gene_dict(ThirdPositiveControl)
FourthPositiveControlDF = flatten_gene_dict(FourthPositiveControl)
FifthPositiveControlDF = flatten_gene_dict(FifthPositiveControl)

In [16]:
len(SecondPositiveControlDF)

6534

In [17]:
# Negative Control List Generation-------
def NegativeControlGeneration(pairedPositiveControlDF, interactingMatrix, flattenGenes):
    Gene1List = pairedPositiveControlDF['Gene1']
    Gene2List = pairedPositiveControlDF['Gene2'].tolist()
    random.shuffle(Gene2List)
    
    # Make a pseudo negative control df 
    pseudoNegPairDF = pd.DataFrame({'Gene1': Gene1List, 'Gene2': Gene2List})
    
    rows_to_drop = []
    for i in range(len(pseudoNegPairDF)):
        gene1 = pseudoNegPairDF.iloc[i]['Gene1']
        gene2 = pseudoNegPairDF.iloc[i]['Gene2']
        gene1_index = flattenGenes.index(gene1)
        gene2_index = flattenGenes.index(gene2)
        
        if interactingMatrix[gene1_index, gene2_index] != 0:
            rows_to_drop.append(i)
    
    pseudoNegPairDF = pseudoNegPairDF.drop(index = rows_to_drop)
            
    return pseudoNegPairDF

In [18]:
FirstNegativeControl = NegativeControlGeneration(FirstPositiveControlDF, matrix, flattenGeneList)
FirstNegativeControl.head(5)

,Gene1,Gene2
0,HDAC4,DLG5
1,HDAC5,MED6
2,BCL6,ARMC8
3,EP300,LPAR2
4,KAT2B,SHLD3


In [19]:
len(FirstNegativeControl)

6365

In [20]:
SecondNegativeControl = NegativeControlGeneration(SecondPositiveControlDF, matrix, flattenGeneList)
ThirdNegativeControl = NegativeControlGeneration(ThirdPositiveControlDF, matrix, flattenGeneList)
FourthNegativeControl = NegativeControlGeneration(FourthPositiveControlDF, matrix, flattenGeneList)
FifthNegativeControl = NegativeControlGeneration(FifthPositiveControlDF, matrix, flattenGeneList)

In [21]:
print(len(SecondNegativeControl), len(ThirdNegativeControl), len(FourthNegativeControl), len(FifthNegativeControl))

6354 6357 6376 6360


In [22]:
# Sanity Check-------

def SanityCheck(NegativeControl):
    
    sameComplexCount = 0
    for i in range(len(NegativeControl)):
        gene1 = NegativeControl.iloc[i]['Gene1']
        gene2 = NegativeControl.iloc[i]['Gene2']
        gene1associatedGenes = associatedGenes[gene1]
        if gene2 in gene1associatedGenes:
            sameComplexCount += 1
            
    return sameComplexCount


In [23]:
print(SanityCheck(FirstNegativeControl), SanityCheck(SecondNegativeControl), SanityCheck(ThirdNegativeControl), SanityCheck(FourthNegativeControl), SanityCheck(FifthNegativeControl))

0 0 0 0 0


In [24]:
os.getcwd()

'/home/wguo'

In [25]:
FirstPositiveControlDF.to_csv('FirstPositiveControl.csv', index = False)
FirstNegativeControl.to_csv('FirstNegativeControl.csv', index = False)

In [26]:
SecondPositiveControlDF.to_csv('SecondPositiveControl.csv', index = False)
SecondNegativeControl.to_csv('SecondNegativeControl.csv', index = False)

In [27]:
ThirdPositiveControlDF.to_csv('ThirdPositiveControl.csv', index = False)
ThirdNegativeControl.to_csv('ThirdNegativeControl.csv', index = False)

In [28]:
FourthPositiveControlDF.to_csv('FourthPositiveControl.csv', index = False)
FourthNegativeControl.to_csv('FourthNegativeControl.csv', index = False)

In [29]:
FifthPositiveControlDF.to_csv('FifthPositiveControl.csv', index = False)
FifthNegativeControl.to_csv('FifthNegativeControl.csv', index = False)

In [30]:
folder_name = 'AllPositiveAndNegativeControl'
os.makedirs(folder_name, exist_ok = True)
files_to_move = [f for f in os.listdir() if f.endswith('.csv')]

for file in files_to_move:
    shutil.move(file, os.path.join(folder_name, file))